# Lesson 08: Markov Decision Processes and Q Learning

## Part 01: Markov Decision Processes


### Summary

This notebook outlines the general concepts of Markov Decision Processes (MDPs) as discussed by Charles and Michael in the video lectures. Much of the content is scraped from the trnascripts of those videos, so you may hear their voices coming through here. 



#### Decision Making  Reinforcement Learning

Differences are between the three types of learning -  supervised, unsupervised and reinforcement. 
 
- ** Supervised learning ** takes the form of _function approximation_ where you're given a bunch of _x_, _y_ pairs (features and labels), and your goal is to find a function **f** that will map some new _x_ to a proper _y_. If the learner is a good one, then the predicted _y_ will be the same as or close to (in some sense) the true _y_. These _x_'s and _y_'s are often vectors.

- ** Unsupervised learning ** takes a bunch of _x_'s and your goal is to find some "function" **f** that gives you a compact description (this is the equivalent of _y_ now) of the set of _x_'s that you've seen. So we call this _clustering_, or _description_ as opposed to function approximation.

- ** Reinforcement learning (RL) **  takes a bunch of data pairs (_x_'s and _z_'s) and learns some function **f** that can generate the _y_'s. In this case, though, the _x_, _y_ and _z_ elements are very different from the the _x_ and _y_ in the other two types of learning. So the first step in RL is to understand what the _x_, _y_ and _z_'s are.

We are going to motivate the definition of these _x_, _y_ and _z_'s with a simple game. The game is adapated from the video lecture (which in turn adapted it from one of the classic texts on AI -- **Artificial Inelligence - A Modern Approach**, _Stuart Russell and Peter Norvig_). 

### The "World" of Charles and Michael - Quiz

In the video lectures, they describe a simple $ 3x4 $ grid-like world as the premise of a game. There are four kinds of cells in this world:
    1. cells that you can move to without anything spectacular happening
    2. cells that are "walled" and you cannot get to them
    3. cells that are spectacularly bad (you LOSE) and the game ends if you land in one of them
    4. cells that are pectacularly good (you WIN) and the game ends if you land in one of them. 
    
The player starts of in one one of the cells (type 1) selectedly randomly -- the _start_ state. The play proceeds by the player selecting one of a set of actions (_up_, _down_, _left_, or _right_) that  moves the player to a different cell. The object of the game is for the player to get to a cell of type 3 so the player can win the game. Also, a player can never all off the grid; if they attempt to do so, they will end up not moving.

Let's switch to using "you" instead of player (much easier for me to use!). In the grid displayed below we represent a $ 4x4 $ world. **X** represents "walled" cells; if you take an action attempting to move into a walled cell, you end up not moving. At every step, the game's controller responds to your actions deterministically, i.e., if you pick _L_ (to go _left_), the controller moves you to a cell to the immediate left of their current position provided that were an available cell (i.e., not a waled cell or a boundary). 


| A | B | C | D | 
 ----- | ------ | ------------ | --------- | ------------- | ----- 
a |   <code>       </code> |  **X**  | <code>       </code>   |  _WIN_  
b |    |  <code>       </code> |    |  _LOSE_ 
c |    | **X**  |  <code>       </code>  |   <code>       </code> 
d | _start_  |   |    |   
  
**Question 1.** In this world, given a _start_ position of d,A (lower left corner), what are the two optimal (shortest) paths to winning the game? How many moves did it take? What is the probability that the player wins the game?

**Answer:**

> _Answer_

In the world described above, I expect you were able to find the path without too much trouble. Think about how you solved this problem. What were the steps you took? What were the decisions you made along the way? Please discuss with your partner and write them down. It may seem a bit silly, but there is a small point to it. What algorithm might you use to solve this for a grid of any dimension where the world followed the same rules (you don't have to write the algorithm, just think of a way you might be able to do it)? 

** Reflection 1** How I solved the path problem

** Answer: **

 > _Answer_

In the next version of the game, we consider the case where the rules of movement have changed slightly. Now, the game's controller _doesn't_ always move you in the direction you wanted to move. Instead, 80% of the time, the controller moves you in the direction, but 20% of the time, it moves you in a direction perpendicular to the one you wanted to take. For example, let's say from the _start_ position (d,A), you wanted to go up. 80% of the time you would go up to (c,A), 10% of the time you would move to (d,B) and 10% of the time you would remain in (d, A) as the controller would try to move you to the right, but there is a boundary there so you end up not moving at all.

** Question 2.** Pick one of the paths from your answer to Q1. What is the probability that you will win the game if you made the exact same sequence of moves? Remember the minor cases.

**Answer:**

In [ ]:
## Code cell -- you can write out all the steps and probabilities and use python as a simple calculator

Here's some food for thought. How easily could you calculate the probability of winning the game? Are you guaranteed to win it every time? What kind of algorithm might you use to find the optimal path to the WIN state? Discuss these with your partner. Write down some of your thoughts (I am not looking for the correct answer).

** Reflection ** 

 > 

#### Making decisions in a stochastic world 

The exercise in the videos (and above) illusrate how it is possible to come up with a static plan when the world is deterministic, but the problem becomes a bit more challenging when there is uncertainty (or stochasticity). We have two options:
 - plan out what we would do in a deterministic world and try to execute the steps of the plan, then every once in a while, see if we've drifted away from where we thought we should be, re-evaluate, draw up a new plan (still assuming a deterministic world)
 - come up with some way to incorporate all of these uncertainties and probabilities so that we never really have to rethink what to do in case something goes wrong and live with the consequences. Some fraction of the time we would end up not achieving our goal of reaching the WIN position, but that we would have the best chance of winning.
 
The second method is called single agent reinforcement learning, or a Markov Decision Process (MDP). 

Let's look back at our reflections on question 1. You may have come up with a different way, but generally, we look at the grid and say there are specific locations and I need to move from one location to the next. We are going to abstract these words and call the location (a grid cell) a **_state_** and the moves we made an **_action_**. These two terms are essential concepts in the world of RL or MDPs. 

Continuing on with adding more words to our thought processes, in solving the grid-world problem, we were in a state (location), took an action and ended up in another state. The grid was laid out for us so we could see the connections between the states and knew that we had to go _up_ from (d, A) to get to (c, A).  Well this map can also be abstracted and is another important concept -- the **_model_** or the **_transition model_**. It is a function of three variables, a state $ s $, an action $a$ and another state (where we end up) $s'$ and it maps those three variables to a number, the probability of the action $a$ taking you from state $s$ to $s'$. Note that this is a conditional probability where the _prior_ conditions are state $s$ and action$a$. This mapping then allows up to incorporate one type of stochasticity (as described for question 2).

**Question 3:** Using notation like (d,A) to represent a state, one of the four letters U, D, L, R to denote the action, write down all possible entries in the transition model for the deterministic world (from question 1) starting in states (b,C) and (c, A). Each state should have four entries. If the move isn't permitted, then the "final" state is the same as the initial state. 

Repeat this for state (c, A) for the stochastic world of question 2, where the action you take doesn't necessarily move you to the state you were expecting to move to.

**Answer:**

 >  

In very broad terms, the colection of states define our world or universe, the actions are the things we can do to explore the world, and the transition model encapsulates the rules of the game.

Turns out that in the simplistic model, we left out explicitly mentioning a few key points. The first one in the _Markovian property_ which essentially states that only the present state that matters. This amounts to saying that the history of how we got there doesn't matter. 

The second one is the _stationarity property_ which assumes that the transition model itself doesn't change. 

The third one is the _fully-observable property_ which implies that we can always know which state we are in. 

These three abstract concepts of **state**, **action** and **model**, subject to the three constraints above define a **Markov Process**. In the Markovian world, as you live (or explore), you can describe your "path through life" as a sequence of states visited in order {$s_1, s_2, ... s_t, s_{t+1}, .. $}. This is called a _Markov chain_.

While these are all we need to define in order to describe how we 'live' in the world,  if we want to have a purpose, we need to add the notion of a **reward**. A **reward** is simply a scalar value that you get for being in a state. It encompasses our domain knowledge; the reward you get from the state tells you the usefulness of entering into that state. Why is this important? Without a reward, there is no reason to be in any state vs. the next and also no real need to make decisions. 

These four things, by themselves, along with this Markov property and non-stationarity, defines a Markov Reward Process. Now, our "path through life" is a bit more complicated. It is still a sequence as before, but each element of the sequence has more information embedded in it $<s, a, R>$. This saying that in principle, you get a reward for every state (but this value could be zero) and we can therefore write down a sequence of rewards {$R_1, R_2, ... R_t, R_{t+1}, .. $}. 

Let's forget the _action_ for the moment and define a few useful functions.

##### Return

In the sequences of states and rewards, we identified the state and reward at step t (a time step or a move). We need to define another function, the _return_, $G_t$, for any state $s_t$ which is the total discounted future reward from $t$ _forward_ until the end of the sequence. 

$$ \large G_t = R_{t+1} + \gamma R_{t+2} +.. = \sum_{k=0} \gamma^k R_{t+k+1} $$

$ \gamma $ is called the _discount rate_ and is a positive real number beteen 0 and 1. It is used to control the "present value of future reward".

##### Value
One call also define a _value_ function, $V(s)$, that is the expected return for starting in state $s$. 

$$ \large V(s) = E[G_t\ \vert\  S_t = s] $$

Without going through the steps of proving it, we'll write down a recursion that expresses _value_ in terms of _reward_:

$$\large V(s) = R(s) + \gamma \sum_{s' \epsilon S} P_{ss'}V(s') $$

where $P_{ss'}$ is the probability of moving from state $s$ to $s'$ in the next step. 

##### Short break from the heavy stuff in the last couple of cells!! 

** Question 4:** I reproduced the "game board" from above and changed the start position so the optimal path appears unambiguous. Use the stochastic rules we used for question 2, i.e., moving up or down results in travelling up or down 80% of the time and moving left or right 10% of the time each. Moving left or right results in travelling left or right 80% of the time and up or down 10% of the time each.

Since we aren't ready to include actions just yet, use the direction you would use in a deterministic world for the shortest path to get the probability of making a transition from one state to another. For example, $P$ for state (a,C) has the following values: $P(.->(a,D)) = 0.8$, $P(.->(a,C)) = 0.1$ and $P(,->(b,C)) = 0.1$. _[The $P(.->x)$ notation is shortened form of saying P((a,C)->x).]_

| A | B | C | D | 
 ----- | ------ | ------------ | --------- | ------------- | ----- 
a |   <code>       </code> |  **X**  | <code>       </code>   |  _WIN_  
b |    |  _start_ |    |  _LOSE_ 
c |    | **X**  |  <code>       </code>  |   <code>       </code> 
d |   | <code>       </code>  |    |   

Assume the following rewards:
 - +0.2 when you end up in a new state that is a non-terminal state and not a walled cell
 - -0.1 if you make a move that leaves you in the same state as you started
 - +5.0 when you end up in the WIN state (game ends)
 - -5.0 when you end up in the LOSE state (game ends).
Assume $ \gamma\ =\ 0.5 $. 

- a) Calculate $V$ for state (a,C). The value of a terminal state is conventionally set to 0. 
- b) Next, calculate $V$ for the state (b,C), (c,C) and (b,B) in that order. If you end up needing a value for a state that you don't have one yet, use 0.
 

In [10]:
# Lets use a list of lists to make tracking numbers easier
V = [[0]*4 for i in range(4)]
def print_matrix(X):
    fmt="{:>8.2f}"*len(X[0])
    for x in X:
        print fmt.format(*x)

## TODO - You need to assign appropriate values to the four elements listed below.
##    Use the recursion formula for V for a MRP given above
#
V[0][2]=0
V[1][2]=0
V[2][2]=0
V[1][1]=0

print_matrix(V)


    0.00    0.00    0.00    0.00
    0.00    0.00    0.00    0.00
    0.00    0.00    0.00    0.00
    0.00    0.00    0.00    0.00


# STOP HERE!!

What follows is notes on terminology and equations. We will review them in the in-class presentation, but I have included them here for your reference.

#### Markov Decision Process

The difference between a Markow Reward Process (MRP) and a Markov Decision Process (MDP) is -- MDP's include decisions. So we are now going to expand the functions/terms we've just defined to include _actions_. Decisions are encapsulated by the _policy_ function $pi(a | s)$ which is defined as a probability distribution over the set of actions ${ a }$ given a state $s$. While we haven't yet figured out how to determine the polic, we see that the _policy_ is the solution to the MDP. 

$$\large \pi(a | s) = P(a_t = a \vert s_t = s) $$

The _value_ (also called the _state-value_) function for an MDP needs to incorporate the policy, and looks very similar to the _value_ defined for a MRP above: 

$$ \large V_{\pi} (s) = E_{\pi}[G_t\ \vert\  S_t = s] $$

The _utility_ function, $U$ which is very similar to the _return_ function for MRPs.  It essentially captures the total discounted future reward.  

We also define the _action-value function_ $ Q_{\pi} (s\ |\ a) $ as the expected return starting from state $s$, taking action $a$, and then following policy $\pi $. 

There are many relations that allow us to express one function in terms of another. For our purpose, the most relevant one is for $Q$:

$$ \large Q_{\pi}(s, a) = R_s^a  + \gamma \sum_{s' \epsilon S}\ P_{ss'}^a\ \sum_{a' \epsilon A}\ \pi(a'|s')Q_{\pi}(s', a') $$


#### Optimal policy

In everything we've learned in the ND so far, we've talked about optimizing some value. 
So it is with MDPs. Having introduced the idea of a reward, the most natural thing to think about optimizing is the reward. We have this sequence of rewards, how do we turn it into a single number? We could just add them. Well, simply adding them poses a problem, and we already described several functions that are better candidates than just rewards. One possible choice is to pick the policy that maximizes the expected value for any state. This is the *optimal policy* $\pi^*$.

 

#### Temporal Difference (TD) Learning

There is a relatively simple iterative algorithm, the TD0 learning algorithm that is very effective. The update equation is:

$$\large  V(s_t) \ = \ V(s_t) + \alpha [R_{t+1} +\gamma V(s_{t+1}) - V(s_t)] $$



The [TD learning algorithm can be applied to Q-learning](https://webdocs.cs.ualberta.ca/~sutton/book/bookdraft2016sep.pdf) following this simple prescription:

- Initialize Q(s,a) for $ \all s \epsilon S$ and $ \all a \epsilon A $ arbitrarily. This is usually a constant value (often 0). Set Q for all terminal states to 0
- For each episode
    - Initialize S, the set of states
    - Repeat for each step
        - Choose action $a$ from policy derived from $Q$
        - Take action $a$, observe reward $R(s')$ and final state $s'$.
        - Update Q using:
        $$ \large Q(s,a) = Q(s,a) + \alpha [R_{s'} + \gamma max_a Q(s', a)\ -\ Q(s,a)] $$
        s <- s'
     until $s$ is terminal
     
 